# Sección XII
# Error handling

Go adopta un enfoque diferente. Para el manejo simple de errores, las devoluciones de valores múltiples de Go facilitan informar un error **sin sobrecargar el valor de retorno**. Un tipo de error canónico, junto con otras características de Go, hace que el manejo de errores sea agradable pero bastante diferente al de otros idiomas.


Go también tiene un par de funciones integradas para señalar y recuperarse de condiciones realmente excepcionales. El mecanismo de recuperación se ejecuta solo como parte del estado de una función que se elimina después de un error, lo cual **es suficiente para manejar una catástrofe pero no requiere estructuras de control adicionales** y, cuando se usa bien, puede resultar en un código de manejo de errores limpio.


Consulte el artículo Aplazar, entrar en pánico y recuperar para obtener más información. Además, la publicación de blog Los errores son valores describe un enfoque para manejar los errores de manera limpia en Go al demostrar que, dado que los errores son solo valores, se puede implementar todo el poder de Go en el manejo de errores.

    - go.dev/doc/faq#exceptions
    - Blog : https://go.dev/blog/errors-are-values
    - https://pkg.go.dev/builtin#error


type error interface{


        Error() string


}

Entonces **error** es un tipo.

So any type that has this method attached to it is also an error and you just have to create a type.

You could create a struct, you could attach a method called error to that struct, it could return a string. Then it will implicitly implement type error and it will be an error.

Write the code with errors before writing the code without errors. Always check for errors.
Always, always, always.*
    

You don't want to get in the habit of just using that underscore character and throwing errors away.    You always want to check your error

In [8]:
import "fmt"

In [11]:
func main() {
    n, err := fmt.Println("Hola")

    if err != nil {
        fmt.Println(err)
    }

    fmt.Println(n)

}
main()

Hola
5


In [2]:
import "io/iountil"

In [4]:
import "os"

In [5]:
func main(){

	f, err :=os.Open("names.txt")


	if err != nil{
		fmt.Println(err)
		return
	}

	defer f.Close()


	bs, err := iountil.ReadAll(f)
	if err != nil{
		fmt.Println(err)
		return
	}

	fmt.Println(strings(bs))
}

ERROR: repl.go:14:13: undefined "iountil" in iountil.ReadAll <*ast.SelectorExpr>

Formas de escribir los errores:
- fmt.Println()
- log.Println()
- log.Fatalln()
    - os.exit()
    - defer function doesn't work
    - Ejemplo cool en error_5.go
       
.

    
- log.Panicln()

    - os.Exit
    - defer function run
    - can use recover
    
- Panic

In [10]:
func main(){
    _, err:=os.Open("nofile.txt")
    
    if err!=nil{
        fmt.Println("Acá hay un error: ",err)
    }
    
    fmt.Print("Salí")
}
main()

Acá hay un error:  open nofile.txt: no such file or directory
Salí

In [6]:
import "log"
func main(){
    _, err:=os.Open("nofile.txt")
    
    if err!=nil{
        //fmt.Println("Acá hay un error: ",err)
        log.Println("log form: ",err) // Si hay error en terminal! identico al anterior
                                      // con un registro de la fecha.
                                      // ejemplo: err_4.go
    }
    
    fmt.Println("Salí")// Si sale!!
}
main()

Salí


In [5]:
Fatal calls os.exit!!
It kills the kernel!!

careful!!

func main(){
    
    _, err:=os.Open("nofile.txt")
    if err!=nil{
        //fmt.Println("Acá hay un error: ",err)
        //log.Println("log form: ",err) 
        
        log.Fatalln(err)
    }
    
    fmt.Print("Salí")
}
main()

ERROR: repl.go:1:7: expected ';', found 'IDENT' calls (and 1 more errors)

# Panic
-https://pkg.go.dev/builtin#panic

The panic built-in function stops normal execution of the current goroutine. When a function F calls panic, normal execution of F stops immediately. Any functions whose execution was deferred by F are run in the usual way, and then F returns to its caller. To the caller G, the invocation of F then behaves like a call to panic, terminating G's execution and running any deferred functions. This continues until all functions in the executing goroutine have stopped, in reverse order. At that point, the program is terminated with a non-zero exit code. This termination sequence is called panicking and can be controlled by the built-in function recover.



In [13]:

func main(){

    defer foo() // En este caso si sale! corren las func defer!!
    
    _, err:=os.Open("nofile.txt")
    if err!=nil{
        //fmt.Println("Acá hay un error: ",err)
        //log.Println("log form: ",err) 
        //log.Fatalln("Puedo escribir algo aca?",err)

        log.Panicln("Puedo escribir algo aca?",err,err.Error()) // equivalente a un frt.println 
                                                    // seguido de un panic

    }
    
    fmt.Print("No salí, veamos si las funciones diferidas corren")   
}


func foo(){
    fmt.Println("funcion diferida")
}

main()

funcion diferida


ERROR: Puedo escribir algo aca? open nofile.txt: no such file or directory open nofile.txt: no such file or directory


In [14]:
package main

import(
    "os" 
    "fmt"
)

func main(){

    defer foo() // Ahora si sale! corren las func defer!!
    
    _, err:=os.Open("nofile.txt")
    if err!=nil{
        //fmt.Println("Acá hay un error: ",err)
        //log.Println("log form: ",err) 
        //log.Fatalln("Puedo escribir algo aca?",err)

        //log.Panicln("Puedo escribir algo aca?",err,err.Error()) 

        //panic("y aca?",err) // En estos casos no puedo mandar un mensaje!
        panic(err) 
    }
    
    fmt.Print("No salí, veamos si las funciones diferidas corren")   
}


func foo(){
    fmt.Println("funcion diferida")
}

main()

funcion diferida


ERROR: open nofile.txt: no such file or directory

# Defer, panic and recover

-https://go.dev/blog/defer-panic-and-recover

A defer statement pushes a function call onto a list. The list of saved calls is executed after the surrounding function returns. Defer is commonly used to simplify functions that perform various clean-up actions.



In [15]:
func CopyFile(dstName, srcName string) (written int64, err error) {
    src, err := os.Open(srcName)
    if err != nil {
        return
    }
    defer src.Close()

    dst, err := os.Create(dstName)
    if err != nil {
        return
    }
    defer dst.Close()

    return io.Copy(dst, src)
}

ERROR: repl.go:15:12: undefined identifier: io

A deferred function’s arguments are evaluated when the defer statement is evaluated.


In [25]:

//In this example, the expression “i” is evaluated when the Println call is deferred.
//The deferred call will print “0” after the function returns.

func main(){
    fmt.Println("defer")
    a()
    fmt.Println("cerrar")
    
    
}
func a() {
    i := 1
    defer fmt.Println("el primero ",i)
    i++
    defer fmt.Println("el segundo ",i)
    return
}
main()

defer
el segundo  2
el primero  1
cerrar


Deferred function calls are executed in Last In First Out order after the surrounding function returns.


In [28]:
func main() {
    for i := 0; i < 4; i++ {
        defer fmt.Println("Caso: ",i)
    }
}
main()

Caso:  3
Caso:  2
Caso:  1
Caso:  0


Deferred functions may read and assign to the returning function’s named return values.


In [35]:
func main() {
    var x int
    x++
    fmt.Println("Caso normal",x)
    
    j := c()
    fmt.Println(j)
}

func c() (i int) {
    defer func() { i++ }()
    return 1
}
main()

Caso normal 1
2


## Panic:
Panic is a built-in function that stops the ordinary flow of control and begins panicking. When the function F calls panic, execution of F stops, any deferred functions in F are executed normally, and then F returns to its caller. To the caller, F then behaves like a call to panic. The process continues up the stack until all functions in the current goroutine have returned, at which point the program crashes. Panics can be initiated by invoking panic directly. They can also be caused by runtime errors, such as out-of-bounds array accesses.

## Recover:

Recover is a built-in function that regains control of a panicking goroutine. Recover is only useful inside deferred functions. During normal execution, a call to recover will return nil and have no other effect. If the current goroutine is panicking, a call to recover will capture the value given to panic and resume normal execution.



In [36]:
func main() {
    f()
    fmt.Println("Returned normally from f.")
}

func f() {
    defer func() {
        if r := recover(); r != nil {
            fmt.Println("Recovered in f", r)
        }
    }()
    fmt.Println("Calling g.")
    g(0)
    fmt.Println("Returned normally from g.")
}

func g(i int) {
    if i > 3 {
        fmt.Println("Panicking!")
        panic(fmt.Sprintf("%v", i))
    }
    defer fmt.Println("Defer in g", i)
    fmt.Println("Printing in g", i)
    g(i + 1)
}
main()

Calling g.
Printing in g 0
Printing in g 1
Printing in g 2
Printing in g 3
Panicking!
Defer in g 3
Defer in g 2
Defer in g 1
Defer in g 0
Recovered in f 4
Returned normally from f.


# Errors with info

erros.New() recibe strings

In [2]:
This example kills the kernel
You better look the error_9.go

import (
	"errors"
	"log"
)

func main() {
    _, err := sqrt(-10)
    if err != nil {
        log.Fatalln(err)
    }
}

func sqrt(f float64) (float64, error) {
    if f < 0 {
        return 0, errors.New("norgate math: square root of negative number") // incluye
                                                                            // esta info
                                                                            // en el error
    }
    return 42, nil
}
main()

ERROR: repl.go:1:6: expected ';', found 'IDENT' example (and 3 more errors)

In [5]:
import "errors"

In [3]:
Mataría el Kernel por el Fatalln

var ErrNorgateMath = errors.New("norgate math: square root of negative number")

func main() {
	fmt.Printf("%T\n", ErrNorgateMath)
	_, err := sqrt(-10)
	if err != nil {
		log.Fatalln(err)
	}
}

func sqrt(f float64) (float64, error) {
	if f < 0 {
		return 0, ErrNorgateMath
	}
	return 42, nil
}

ERROR: repl.go:1:10: expected ';', found 'IDENT' el

In [9]:

var ErrNorgateMath = errors.New("norgate math: square root of negative number")

func main() {
    fmt.Printf("%T\n", ErrNorgateMath)
}

main()

*errors.errorString


In [10]:
Tambien problemas con el kernel
ver error_10.go

import (
    "fmt"
    "log"
)

func main() {
    _, err := sqrt(-10.23)
    if err != nil {
        log.Fatalln(err)
    }
}

func sqrt(f float64) (float64, error) {
    if f < 0 {
        return 0, fmt.Errorf("norgate math again: square root of negative number: %v", f)
        // Basicamente es como llamar New() pero dandole un formato 
    }
    return 42, nil
}

ERROR: repl.go:1:9: expected ';', found 'IDENT' problemas (and 3 more errors)

In [ ]:
Tambien podemos asignar una variable!

func main() {
    _, err := sqrt(-10.23)
    if err != nil {
        log.Fatalln(err)
        }
}

func sqrt(f float64) (float64, error) {
    if f < 0 {
        ErrNorgateMath := fmt.Errorf("norgate math again: square root of negative number: %v", f)
        return 0, ErrNorgateMath
    }
    return 42, nil
}

Podemos crear structuras con el método Error() atadas a este. Ver error_11.go

- https://pkg.go.dev/builtin#error

Todas las estructuras a las que cree con el metodo Error() serán tambien tipo error